In [9]:
from pls_es import *
nb_objectif = 2
file = "2KP50-TA-0.dat"
w = (0.2,0.8)
# w = [0.2,0.1,0.2,0.3,0.15,0.05]
infos = read_file(file,nb_objectif)


In [10]:
print("Solu_random:",get_v_total(infos,init_p(infos)[0],w = w,model = 'ponderee'))

Solu_random: 13495.0


In [17]:
pareto_index = PLS(file,nb_objectif = nb_objectif)
print(len(pareto_index)," solutions")
for s in pareto_index:
    # print(s)
    # print(get_v_total(infos,s,w = (0.2,0.8),model = 'ponderee'))
    # print(get_v_total(infos,s,w = (0.2,0.8),model = 'OWA'))
    pass

26  solutions


In [24]:
pareto_index = PLS(file,nb_objectif = nb_objectif)
print(len(pareto_index)," solutions")
for s in pareto_index:
    # print(s)
    # print(get_v_total(infos,s,w = (0.2,0.8),model = 'ponderee'))
    # print(get_v_total(infos,s,w = (0.2,0.8),model = 'OWA'))
    pass
res = procedure1(infos,pareto_index,w = w,model = 'ponderee')
print("Procedure1:",get_v_total(infos,res,w = w,model = 'ponderee'))


27  solutions
Procedure1: 19069.0


In [32]:
pareto_index = PLS_EI(w,'ponderee',file,nb_objectif = nb_objectif)
print("Procedure2:",get_v_total(infos,pareto_index,w = w,model = 'ponderee'))

Procedure2: 12813.000000000002


In [34]:
pareto_index = PLS_EI(w,'OWA',file,nb_objectif = nb_objectif)
print("Procedure2:",get_v_total(infos,pareto_index,w = w,model = 'OWA'))

Procedure2: 14600.0
